# Procesamiento Digital de Imágenes Satelitales
# Instituto Gulich - Mayo - Julio 2025

---
#  Clase 06b: Un sitio de estudio. ROIS y Máscaras
---


### Objetivos de la notebook:

- En esta notebook esperamos que aprendas a mirar en conjunto datos raster y ROIS vectoriales.

### Datos con los que trabajaremos en esta Notebook:

- La imagen Sentinel-2 del Dique Roggero en formato original comprimido 
    - **L1C_T21HUB_A017181_20181006T135109.zip**
    - **L1C_T21HUB_A023044_20191120T135113.zip**
- Capa vectorial
    - **zona_dique.geojson**
    - **rois_dique.geojson**

### Las librerías que vamos a manejar son:

```python
import os

import numpy as np
import matplotlib.pyplot as plt

import rasterio
import rasterio.mask

from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask

import geopandas as gpd

import pandas as pd
import seaborn as sns

import sys
sys.path.append('../')

from funciones import *

```

vamos a definir primero nuestras rutas:

```python
data_dir = '../data/raw_data/'
path_proc = '../data/proc/'
path_out = '../data/out/'
path_vector = '../data/raw_data/vector/'
```

Utilicemos el vector "zona_dique.geojson" para recortar nuestras capas de entrada:
```python
mbb_dique = compute_mbb(os.path.join(path_vector, 'zona_dique.geojson'), grid_step=100)
```

Utilizando la función creada en la notebook anterior, vamos a leer nuestros datos:

```python
fn_zip = 'L1C_T21HUB_A017181_20181006T135109.zip'
img, crs, gt = extract_10m_bands_Sentinel2_ZIP(os.path.join(data_dir, fn_zip),
                                                             mbb=mbb_dique)
```

Guardemos nuestro _stack_ para más adelante:
```python
bandnames_B_G_R_NIR_NDVI = ['Blue', 'Green', 'Red', 'NIR', 'NDVI']
guardar_GTiff(os.path.join(path_proc, 'S2_dique_20181006.tif'),
              crs,
              gt,
              img,
              bandnames=bandnames_B_G_R_NIR_NDVI)
```

Veamos nuestro recorte:
```python
fig, ax = plt.subplots(1, 2, figsize=(12, 10), sharey=True)

show(nequalize(img[[2, 1, 0]]), transform=gt, ax=ax[0])
show(nequalize(img[[4, 3, 0]]), transform=gt, ax=ax[1])

ax[0].set_title('Combinación Color Verdadero')
ax[1].set_title('Combinación Falso Color: NDVI, NIR, BLUE')
ax[0].ticklabel_format(style='plain')
ax[1].ticklabel_format(style='plain')
plt.tight_layout()
plt.show()
```

## Tratemos de hacer una máscara de agua

```python
NDVI = img[4]
NDVI < 0.0
```

```python
fig, ax = plt.subplots(figsize=(10, 10))
show(NDVI < 0.25, transform=gt, ax=ax)
ax.ticklabel_format(style='plain')
plt.show()
```

## Veamos un poco los histogramas

rasterio también nos permite graficar de manera sencilla los histogramas de nuestros datos mediante una función llamada [show_hist](https://rasterio.readthedocs.io/en/stable/api/rasterio.plot.html#rasterio.plot.show_hist). Veamos la distribución de los datos de nuestras bandas:

```python
fig, ax = plt.subplots(1, 1)
ax.set_xlim(xmin=0.025, xmax=0.225)
show_hist(
    img[[2, 1, 0]], bins=1024, lw=0.0, alpha=0.5, ax=ax,
    histtype='stepfilled',
    title="Histogram RGB",
    label=['R', 'G', 'B'])

plt.show()
```

Veamos el histograma de nuestra capa de NDVI

```python
fig, ax = plt.subplots()

show_hist(
    NDVI, bins=256, lw=0.0, alpha=1, ax=ax,
    histtype='stepfilled',
    title="Histogram NDVI",
    label=['NDVI'])

plt.show()
```

## ¿a qué corresponde cada moda?
Podríamos generar manualmente umbrales para identificar qué es cada una de estas modas. Por ejemplo, podríamos identificar visualmente los siguientes cortes:

```python
vmin = NDVI.min()
thr_1 = 0
thr_2 = 0.35
vmax = 1

fig, ax = plt.subplots()

show_hist(
    NDVI, bins=256, lw=0.0, alpha=1, ax=ax,
    histtype='stepfilled',
    title="Histogram NDVI",
    label=['NDVI'])

ax.axvline(thr_1)
ylim = ax.get_ylim()
ax.axvline(thr_2)
ax.fill_betweenx(ylim, vmin, thr_1, facecolor='green', alpha=0.1)
ax.fill_betweenx(ylim, thr_1, thr_2, facecolor='blue', alpha=0.1)
ax.fill_betweenx(ylim, thr_2, vmax, facecolor='orange', alpha=0.1)
ax.set_ylim(*ylim)
ax.set_xlim(NDVI.min(), NDVI.max())

plt.show()
```

```python
fig, ax = plt.subplots(1, 2, figsize=(12, 10), sharey=True)

show(NDVI < thr_1, ax=ax[0], transform=gt)
show(nequalize(img[[2, 1, 0]]), transform=gt, ax=ax[1])

ax[0].ticklabel_format(style='plain')
ax[1].ticklabel_format(style='plain')

ax[0].set_title(f'Valores de NDVI menores a {thr_1}')
ax[1].set_title('Combinacion Color Verdadero')

plt.tight_layout()
plt.show()
```

```python
fig, ax = plt.subplots(1, 2, figsize=(12, 10), sharey=True)

show((NDVI > thr_1) & (NDVI < thr_2), ax=ax[0], transform=gt)
show(nequalize(img[[2, 1, 0]]), transform=gt, ax=ax[1])

ax[0].ticklabel_format(style='plain')
ax[1].ticklabel_format(style='plain')

ax[0].set_title(f'Valores de NDVI entre {thr_1} y {thr_2}')
ax[1].set_title('Combinacion Color Verdadero')

plt.tight_layout()
plt.show()
```

```python
fig, ax = plt.subplots(1, 2, figsize=(10, 10), sharey=True)

show(NDVI > thr_2, ax=ax[0], transform=gt)
show(nequalize(img[[2, 1, 0]]), transform=gt, ax=ax[1])

ax[0].ticklabel_format(style='plain')
ax[1].ticklabel_format(style='plain')

ax[0].set_title(f'Valores de NDVI mayores a {thr_2}')
ax[1].set_title('Combinacion Color Verdadero')

plt.tight_layout()
plt.show()
```

## Veamos ahora el histograma para NIR

```python
fig, ax = plt.subplots(figsize=(12, 8))

ax.set_xlim(xmin=0.0, xmax = 0.6)
show_hist(img[-2], bins=512, lw=0.0, alpha=1, ax=ax,
          histtype='stepfilled',
          title="Histogram NIR",
          label=['NIR'])

plt.show()
```

### Ejercicio 8.1 de máscaras:
A partir de lo anterior, responder:

- ¿A qué corresponde cada una de las dos modas?
- A partir de los histogramas, seleccionar un umbral en NIR y NDVI para realizar máscaras de agua. ¿Cuál delimita mejor el cuerpo de agua?
- Usando un recorte similar al S2_dique_20181006.tif pero para la imagen del 2019-11-20, generár una máscara de agua con el umbral adecuado para esta nueva fecha. Comparar ambas máscaras. ¿A qué corresponden los pixels marcados como agua en solo una de las imagenes?
- Armar una imagen que tenga valor 0 si no hay agua, 1 si hay agua en una de ambas fechas y 2 si hay agua en ambas fechas. Plotear el resultado.


## Hasta acá miramos la imágen entera.
## Agreguemos ahora información (etiquetas) sobre sus pixels.

```python
rois_shp = gpd.read_file(os.path.join(path_vector, 'rois_dique.geojson'))
rois_shp.head(10)
```

Veamos qué información tenemos:

```python
# Veamos las clases de nuestros ROI
rois_shp['clase']
```

```python
rois_shp['geometry']  # series of shapely geometries
```

```python
print(rois_shp.crs)
```

```python
fig, ax = plt.subplots(figsize=(12, 12))
rois_shp.plot(ax=ax)
ax.ticklabel_format(style='plain')
plt.show()
```

## Realicemos la superposición de nuestros ROIs en nuestro NDVI

```python
fig, ax = plt.subplots(figsize=(12,12))
rois_shp.boundary.plot(ax=ax, color='orangered')
show(NDVI, transform=gt, ax=ax)
ax.ticklabel_format(style='plain')
plt.show()
```

Podríamos filtrar por clase:

```python
fig, ax = plt.subplots(figsize=(12, 12))
rois_shp[rois_shp['clase']=='urbano'].boundary.plot(ax=ax, color='orangered')
show(NDVI, transform=gt, ax=ax)
ax.set_title('ROIS de la zona urbana')
ax.ticklabel_format(style='plain')
plt.show()
```

o colorear por clase:

```python
fig, ax = plt.subplots(figsize=(12, 12))
rois_shp.plot(ax=ax, column='clase')
rois_shp.boundary.plot(ax=ax, color='orangered')
show(NDVI, transform=gt, ax=ax)
ax.ticklabel_format(style='plain')
plt.show()
```

### Ejercicio 8.2 Visualización de imágenes (opcional):
1. Rehacer el gráfico usando de fondo la imagen RGB equalizada y ploteando los bordes de los ROIS.
1. Rehacer el gráfico pero ploteando solo los ROIs de clase 'rural'.